In [1]:
%load_ext autoreload
%autoreload 2

In [2]:
%matplotlib widget
import matplotlib.pyplot as plt
import seaborn as sns

In [3]:
import numpy as np
import pandas as pd
import xarray as xr

In [4]:
xr.set_options(keep_attrs=True)

In [5]:
import ecephys_analyses as ea
import ecephys.plot as eplt
from ecephys.utils.xarray import get_smoothed_ds

In [6]:
from ecephys.scoring import get_timepoint_states

def filter_states(ds, hypnogram, states):
    timepoint_states = get_timepoint_states(hypnogram, ds.time)
    return ds.where(np.isin(timepoint_states, states))

In [7]:
def load_condition_data(subject, experiment, condition, power_as_percentage_of=None):
    hypnogram = ea.load_hypnogram(subject, experiment, condition)    
    spws = ea.load_spws(subject, experiment, condition, condition_start_dt=hypnogram.start_time.min())
    bp = ea.load_bandpower(subject, experiment, condition)
    if power_as_percentage_of:
        bp = bp / power_as_percentage_of * 100
    
    bp = get_smoothed_ds(bp, smoothing_sigma=10, in_place=True)
    
    return hypnogram, spws, bp

In [63]:
def plot_condition(hypnogram, spws, power):

    fig, (mpta_swa_ax, sr_swa_ax, density_ax, duration_ax, amplitude_ax, integral_ax) = plt.subplots(6, 1, figsize=(22,12))
    xmargin=0
    
    mpta_swa_ylabel = 'CTX delta, WM ref'
    sr_swa_ylabel = 'SR delta, WM ref'
    #mpta_swa_ylabel = mpta_swa_ylabel + ', \n % BSL NREM mean'
    #sr_swa_ylabel = sr_swa_ylabel + ', \n % BSL NREM mean'

    # units = (uV^2/Hz)
    sns.lineplot(x=power.datetime.values, y=power.mpta_delta_wm_ref.values, color='black', ax=mpta_swa_ax)
    mpta_swa_ax.set(xlabel=None, ylabel=mpta_swa_ylabel, ylim=MPTA_SWA_LIM, xmargin=xmargin, xticks=[])

    sns.lineplot(x=power.datetime.values, y=power.sr_delta_wm_ref.values, color='black', ax=sr_swa_ax)
    sr_swa_ax.set(xlabel=None, ylabel=sr_swa_ylabel, ylim=SR_SWA_LIM, xmargin=xmargin, xticks=[])
    
    mpta_swa_ax.axhline(100, color='r', linestyle=':')
    sr_swa_ax.axhline(100, color='r', linestyle=':')

    binwidth = 10
    nbins = np.int((spws.start_time.max() - spws.start_time.min()).total_seconds() / binwidth)
    sns.histplot(data=spws, x='start_time', bins=nbins, color='black', ax=density_ax)
    density_ax.set(xlabel=None, ylabel='Frequency (Hz)', ylim=DENSITY_LIM, xmargin=xmargin, xticks=[])

    sns.scatterplot(data=spws, x="start_time", y="duration", s=2, ax=duration_ax)
    duration_ax.set(xlabel=None, ylabel="Duration (s)", ylim=DURATION_LIM, xmargin=xmargin, xticks=[])

    sns.scatterplot(data=spws, x='start_time', y='sink_amplitude', s=2, ax=amplitude_ax)
    amplitude_ax.set(xlabel=None, ylabel="Sink amplitude (mA/mm)", ylim=AMPLITUDE_LIM, xmargin=xmargin, xticks=[])

    sns.scatterplot(data=spws, x='start_time', y='sink_integral', s=2, ax=integral_ax)
    integral_ax.set(xlabel="Event time (month-date hour)", ylabel="Sink integral (mA * ms)", ylim=INTEGRAL_LIM, xmargin=xmargin)

    for ax in (mpta_swa_ax, sr_swa_ax, density_ax, duration_ax, amplitude_ax, integral_ax):
        eplt.plot_hypnogram_overlay(hypnogram, ax=ax)

    plt.tight_layout()

In [9]:
subject = "Doppio"
experiment = "ketamine-120mg"
condition = "all"

In [10]:
MPTA_SWA_LIM = SR_SWA_LIM = DENSITY_LIM = DURATION_LIM = AMPLITUDE_LIM = INTEGRAL_LIM = (None, None)

In [ ]:
(hypnogram, spws, power) = load_condition_data(subject, experiment, condition)

In [64]:
plot_condition(hypnogram, spws, power)

Canvas(toolbar=Toolbar(toolitems=[('Home', 'Reset original view', 'home', 'home'), ('Back', 'Back to previous …

In [11]:
hypnogram = ea.load_hypnogram(subject, experiment, condition)    

In [12]:
spws = ea.load_spws(subject, experiment, condition, condition_start_dt=hypnogram.start_time.min())

In [55]:
bp = ea.load_bandpower(subject, experiment, condition)

In [56]:
new_coords = bp.datetime.values.min() + pd.to_timedelta(np.arange(bp.time.size) * 3, 's')

In [57]:
bp = bp.assign_coords({"datetime": ("time", new_coords)})

In [61]:
power = get_smoothed_ds(bp, smoothing_sigma=10)